In [ ]:
!pip install numpy matplotlib opencv-python mediapipe scipy scikit-learn tensorflow

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [6]:
import os
os.listdir('/content/drive/MyDrive/DataSet/Aiza')

['Aiza-90_l.MOV',
 'Aiza-90_r.MOV',
 'Aiza-45_r.MOV',
 'Aiza-45_l.MOV',
 'Aiza-00_f.MOV']

In [13]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft
import os
Dataset_path='/content/drive/MyDrive/DataSet'
for folder in os.listdir(Dataset_path):
  folder_path=os.path.join(Dataset_path,folder)
  for file_path in os.listdir(folder_path):
      # Get the last folder name
      last_folder_name = os.path.basename(folder_path)
      # Create a subfolder based on the last folder name
      output_dir = os.path.join("/content/drive/MyDrive/Results", last_folder_name)
      os.makedirs(output_dir, exist_ok=True)

      # Initialize MediaPipe Pose
      mp_pose = mp.solutions.pose
      pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

      # Open the video file
      cap = cv2.VideoCapture(os.path.join(folder_path, file_path))

      # Prepare a list to store the y-coordinates of the right ankle over time
      ankle_positions_y = []

      with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
          while cap.isOpened():
              success, frame = cap.read()
              if not success:
                  break

              # Convert the BGR image to RGB and process it with MediaPipe
              frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
              results = pose.process(frame_rgb)

              # Extract the right ankle position if it's visible
              if results.pose_landmarks:
                  right_ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
                  if right_ankle.visibility > 0.5:  # Ensure the ankle is clearly visible
                      ankle_positions_y.append(right_ankle.y * frame.shape[0])  # Convert normalized position to pixel

      cap.release()

      # Ensure there are data points before proceeding to FFT
      if len(ankle_positions_y) > 0:
          # Convert the list to a numpy array
          ankle_positions_y = np.array(ankle_positions_y)
          # Subtract the mean to center the signal
          centered_ankle_positions_y = ankle_positions_y - np.mean(ankle_positions_y)
          # Apply FFT
          freq_domain = fft(centered_ankle_positions_y)

          # Assuming 30 FPS for the video
          fps = 30
          N = len(centered_ankle_positions_y)
          T = 1.0 / fps
          xf = np.linspace(0.0, 1.0 / (2.0 * T), N // 2)

          # Get the name of the input video file without the extension
          video_title = os.path.splitext(os.path.basename(os.path.join(folder_path, file_path)))[0]

          # Update code to save output graph with the same name as the input video
          output_file = f"{video_title}.png"
          output_path = os.path.join(output_dir, output_file)

          plt.figure(figsize=(10, 6))
          plt.plot(xf, 2.0 / N * np.abs(freq_domain[0:N // 2]))
          plt.title('Frequency Domain Representation')
          plt.xlabel('Frequency (Hz)')
          plt.ylabel('Amplitude')
          plt.savefig(output_path)  # Save the graph as an image file
          plt.close()  # Close the plot to release resources

          print("Output saved successfully:", output_path)
      else:
          print("No ankle positions were extracted. Please check the video and pose detection parameters.")


Output saved successfully: /content/drive/MyDrive/Results/Aiza/Aiza-90_l.png
Output saved successfully: /content/drive/MyDrive/Results/Aiza/Aiza-90_r.png
Output saved successfully: /content/drive/MyDrive/Results/Aiza/Aiza-45_r.png
Output saved successfully: /content/drive/MyDrive/Results/Aiza/Aiza-45_l.png
Output saved successfully: /content/drive/MyDrive/Results/Aiza/Aiza-00_f.png
Output saved successfully: /content/drive/MyDrive/Results/Sawera/Sawera-90_l.png
Output saved successfully: /content/drive/MyDrive/Results/Sawera/Sawera-90_r.png
Output saved successfully: /content/drive/MyDrive/Results/Sawera/Sawera-45_r.png
Output saved successfully: /content/drive/MyDrive/Results/Sawera/Sawera-45_l.png
Output saved successfully: /content/drive/MyDrive/Results/Sawera/Sawera-00_f.png
